# Deep Neural Networks with Different Activation Functions for Breast Cancer Prediction

The purpose of this project is to apply deep neural networks for the breast cancer prediction. One of the hyperparameter will be fined tuned, which is the type of activation functions between hidden layers. The activation functions that will be tuned in this project are ReLu, sigmoid, and tanh functions and see which activation functions yield to the best prediction of the breast cancer.

The data used in this project was taken from the Wisconsin Breast Cancer datasets, available at https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29. Overall, there are 569 examples of tumor with different size of radius, texture, concavity, smoothness, and compactness available.

First, let's import all of the necessary libraries for this project.

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd

## Neural Networks Model

Before starting the calculation for the neural networks model, first let's look at the neural networks schema that will be applied in this project. The neural networks will have three layers: two hidden layers and one output layers. The first hidden layer consists of 5 hidden units and the second hidden layer consists of 3 hidden units, while the output layer consists of 1 output unit.

Below is the schema of neural networks model.

<img src="schema_DNN.png" width="700" height="200">

## Weight Initialization of the Neural Networks

For the neural networks to learn optimally, initializing the weight randomly is very important. If the weight of each hidden units is asisgned to 0, then the hidden layers would basically useless. Hence, let's initialize the parameters randomly.

In [37]:
def initializeParameters (layerDimension):
    
    np.random.seed(3)
    parameters = {}
    L = len(layerDimension)     

    for l in range(1, L):
       
        parameters['W' + str(l)] = np.random.randn(layerDimension[l],layerDimension[l-1])*0.01
        parameters['b' + str(l)] = np.zeros((layerDimension[l],1))

        
        assert(parameters['W' + str(l)].shape == (layerDimension[l], layerDimension[l-1]))
        assert(parameters['b' + str(l)].shape == (layerDimension[l], 1))

        
    return parameters

Here in the code above, $W$ represents weight in each of the hidden units and $b$ represents the bias. The $W$ is multiplied by 0.01 to make sure that the weight will be a small number to make the optimization process run faster. The relationship between $W$, $b$, and the training variable $X$ can be described as below:

$$ W*X+b $$

After define the function to define the weight, now let's move forward to forward propagation algorithm.

## Forward Propagation 

The first step of forward propagation algorithm is to compute the function $Z$ before the application of activation functions. The formula to compute this linear forward algorithm is:

$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}\tag{4}$$

where $[l]$ denotes the number of layers and $A$ is the variable in each of the hidden unit in each hidden layer. This means that for $A^{[0]} = X$

In [3]:
def linearForward(A, W, b):

    Z = np.dot(W,A)+b
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

In the code above, there is a variable called `cache`. This`cache` will basically store the value of $A$, $W$, and $b$ because all of these values will be used again later in the backpropagation algorithm.

Next, we need to compute linear activation forwards, in which we turn the $Z$ value that we got into the result of selected activation functions. In this project, the activation functions that will be used including sigmoid function, ReLu function, and tanh function. The formula of each function can be seen in the following:

$$ sigmoid: \sigma(Z) = \sigma(W A + b) = \frac{1}{ 1 + e^{-(W A + b)}}$$ 
$$ ReLu: A = RELU(Z) = max(0, Z)$$
$$ tanh: \sigma(Z) = \sigma(W A + b) = \frac{e^{(W A + b)}-e^{-(W A + b)}}{e^{(W A + b)}+e^{-(W A + b)}}$$

So, let's define a function for each of this activation function.

In [4]:
def sigmoid(Z):
    
    A = 1/(1+np.exp(-Z))
    cache = Z
    
    return A, cache

In [5]:
def relu(Z):
    
    A = np.maximum(0,Z)
    
    assert(A.shape == Z.shape)
    
    cache = Z 
    return A, cache

In [6]:
def tanh(Z):
    
    A = np.tanh(Z)
    cache = Z 
    
    return A, cache

The cache above will store the value of $Z$, in which will be needed again whenever we compute the backpropagationa nd the gradient of each activation function.

Now let's build a function that call the function to compute the $Z$ as well as turning $Z$ into new value after the application of activation functions.

In [9]:
def activationForward(A_prev, W, b, activation):

    if activation == "sigmoid":

        Z, linear_cache = linearForward(A_prev,W,b)
        A, activation_cache = sigmoid(Z)
     
    elif activation == "relu":

        Z, linear_cache = linearForward(A_prev,W,b)
        A, activation_cache = relu(Z)
    
    elif activation == "tanh":
        
        Z, linear_cache = linearForward(A_prev,W,b)
        A, activation_cache = tanh(Z)
        
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

In the code above, the cache will stor two different types of cache, one for linear cache, which consists of parameter W, A, and b, and the other one is activation cache, which consists parameter Z. These caches will be useful for backpropagation algorithm.

Now, the computation for forward propagation is basically done. However, the computation so far is only valid in one layer. If we want to build deep neural networks with many hidden layers, then we need to do the steps above several times depending on the number of layers in the neural networks. Let's define a function to do this.

In [72]:
def modelForwardProp(X, parameters,activation):
    
    caches = []
    A = X
    L = len(parameters) // 2                 
 
    for l in range(1, L):
        A_prev = A 
        
        A, cache = activationForward(A_prev,parameters["W" + str(l)],parameters["b" + str(l)],activation)
        
        caches.append(cache)
        
    AL, cache = activationForward(A,parameters["W" + str(L)],parameters["b" + str(L)],'sigmoid')
    caches.append(cache)

    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches

Next, let's compute the cost function for the deep neural networks.

## Cost Function

The cost function for deep neural networks is the same as the one in logistic regression. It can be expressed as the following formula:

$$J = -\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(A^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- A^{[L](i)}\right))$$

Now let's define a function to compute the cost based on the formula above.

In [41]:
def computeCost(AL, Y):
    
    m = Y.shape[1]
    
    cost = -1/m*np.sum(np.multiply(Y,np.log(AL))+np.multiply(1-Y,np.log(1-AL)))

    cost = np.squeeze(cost) 
    #assert(cost.shape == ())
    
    return cost

After we define the cost function, now finally let's start with backpropagation algorithm.

## Back Propagation

The idea of backpropagation is to compute the gradient of cost function with respect to each parameters. As the first step, let's define a linear backward step. In forward propagation, the linear step can be defined as:

$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}\tag{4}$$

In backpropagation, we need to compute the derivative of $W$, $A$, and $b$, which are $dW$, $dA$, and $db$. To do this, we use the following formula:

$$ dW^{[l]} = \frac{1}{m} dZ^{[l]} A^{[l-1] T}$$
$$ db^{[l]} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}$$
$$ dA^{[l-1]} =  W^{[l] T} dZ^{[l]}$$

In [12]:
def linearBackward(dZ, cache):
   
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = 1/m*np.dot(dZ,A_prev.T)
    db = 1/m*np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T,dZ)
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In the code above, the linear cache that alrady defined in the `linearForward` function, which consists of parameter A, W, and b will be used to compute their derivative based on the formula above.

Next, let's compute the derivative of $Z$, which is $dZ$ using the gradient of the selected activation functions with the formula:

$$dZ^{[l]} = dA^{[l]} * g'(Z^{[l]})$$

where $ g'(Z^{[l]})$ is the gradient of the activation functions.

In [47]:
def reluBackward(dA, cache):
    
    Z = cache
    dZ = np.array(dA, copy=True) 
    
    dZ[Z <= 0] = 0
    
    assert (dZ.shape == Z.shape)
    
    return dZ

In [48]:
def sigmoidBackward(dA, cache):

    Z = cache
    
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    
    assert (dZ.shape == Z.shape)
    
    return dZ

In [91]:
def tanhBackward(dA, cache):
    
    Z = cache
    
    dZ = dA*(1-np.power(Z,2))
    
    assert (dZ.shape == Z.shape)
    
    return dZ

In the code above, we use the activation cache that already stored during the calculation of the sigmoid function in the forward propagation. As the result, the derivative of $Z$, $dZ$ can be computed. 

Next, let's define a function to call the function above and calculate the $dZ$ and then $dW$, $dA$, and $dB$.

In [50]:
def activationBackward(dA, cache, activation):

    linear_cache, activation_cache = cache
    
    if activation == "relu":
        
        dZ = reluBackward(dA, activation_cache)
        dA_prev, dW, db = linearBackward(dZ, linear_cache)
        
    elif activation == "sigmoid":
       
        dZ = sigmoidBackward(dA, activation_cache)
        dA_prev, dW, db = linearBackward(dZ, linear_cache)
     
    elif activation == "tanh":
       
        dZ = tanhBackward(dA, activation_cache)
        dA_prev, dW, db = linearBackward(dZ, linear_cache)
        
    return dA_prev, dW, db

Now that we define the steps of back propagation in one layer, then we need to create function that will compute the back propagation depending on the number of layers that we defined, just like the one that already defined in forward propagation.

However, we need to calculate the derivative of $A$, $dA$ as the initialization of the back propagation algorithm. This derivative of $dA$ can be computed by the partial derivative of loss function with respect to $A$ at the output layer (or the $Y_{hat}$). The formula of $dA$ is as follows:


$$ dA = - \frac{Y^{[L]}}{A^{[L]}}+\frac{(1-Y^{[L]})}{(1-A^{[L]})} $$

In [71]:
 def modelBackProp(AL, Y, caches, activation):   
    
    grads = {}
    L = len(caches) 
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    # Initializing the backpropagation
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = activationBackward(dAL, current_cache, 'sigmoid')

    for l in reversed(range(L-1)):
        
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = activationBackward(grads["dA"+str(l+1)], current_cache, activation)
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp 
        grads["db" + str(l + 1)] = db_temp

    return grads

The output of the code above is the value for $dW$, $dA$, and $db$, which we can finally use to update the corresponding variables in each iterations of gradient descent optimization. The formula for updating the parameter in gradient descent can be computed using the following formula:

$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} \tag{16}$$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} \tag{17}$$

where $\alpha$ is the learning rate of the gradient descent, the parameter that we need to tune as well. Next, let's define a function to update the parameters based on the formula above.

In [18]:
def updateParameters(parameters, grads, learning_rate):
    
    L = len(parameters) // 2 

    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - (learning_rate*grads["dW"+str(l + 1)])
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - (learning_rate*grads["db"+str(l + 1)])

    return parameters

## Loading Datasets

Now we already complete all of the functions necessary in order to do the whole loop of forward propagation and backward propagation. Next, let's load the datasets necessary for this project.

In [20]:
np.random.seed(1)
trainX = pd.read_csv('trainX.csv')
trainY = pd.read_csv('trainY.csv', names = ['diagnosis'])
testX = pd.read_csv('testX.csv')
testY = pd.read_csv('testY.csv', names = ['diagnosis'])

Here the training and test data are already splitted into the proportion of 70%-30%. Each feature of the data has been normalized as well using min-max approach. Hence, the data now is ready to be used for classification study.

Let's take a look first at the size of the training data and test data.

In [21]:
print('Shape of the training set: '+str(trainX.shape))
print('Shape of the test set: '+str(testX.shape))

Shape of the training set: (398, 30)
Shape of the test set: (171, 30)


Next, we can see the first five rows of training set 

In [22]:
trainX.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,0.607175,0.420697,0.595743,0.473595,0.412386,0.255567,0.346532,0.472068,0.263636,0.084035,...,0.689790,0.502665,0.679267,0.543846,0.528495,0.279138,0.429073,0.820619,0.237138,0.138463
1,0.494060,0.536016,0.488632,0.341251,0.433059,0.292068,0.394096,0.327883,0.125253,0.183235,...,0.360726,0.427772,0.348573,0.205417,0.350855,0.147481,0.223882,0.377663,0.007491,0.086187
2,0.317052,0.223876,0.303849,0.183245,0.362372,0.163088,0.041050,0.093439,0.288384,0.244103,...,0.281750,0.218017,0.254943,0.144564,0.364723,0.125263,0.096326,0.299107,0.244628,0.149416
3,0.273984,0.395671,0.264184,0.154358,0.314706,0.143028,0.072915,0.142346,0.320202,0.271904,...,0.207044,0.305970,0.192390,0.096908,0.149970,0.060628,0.041422,0.164021,0.121033,0.089663
4,0.333617,0.390260,0.317877,0.195080,0.343685,0.153580,0.034255,0.094235,0.230808,0.176706,...,0.263252,0.486674,0.238358,0.130333,0.379912,0.120315,0.049768,0.273643,0.130298,0.138594


And the first five rows of diagnosis training set

In [23]:
trainY.head()

,diagnosis
0,1
1,1
2,0
3,0
4,0


Next, we can define all of the constant parameters that we will need in order to run the gradient descent like the learning rate $\alpha$, the number of iterations, as well as the number of hidden layers and the hidden inputs.

In [43]:
trainY = trainY.values
trainY = trainY.reshape((len(trainY),1))

testY = testY.values
testY = testY.reshape((len(testY),1))

In [76]:
layerDimension = [30, 5, 3, 1] #4 layers with 2 hidden layers
learningRate = 0.0075
numIterations = 200010

## Metrics Definition

Since the classification that will be conducted is about the severity of the tumor: whether the tumor is benign ot malignant, then the accuracy metrics would not be the best metrics for this condition. The emphasize should be focused in supressing the amount of false negative in the algorithm because the cost of having miss-classified an actual positive will be huge. 

Just imagine where there is a patient that has a breast tumor and we make a decision that the tumor is benign when in fact it is malignant. This situation will endanger the patient life and thus, creating a machine learning algorithm that can supress the amount of false negative should be a priority.

Because of this, instead of accuracy or Jaccard index, the metrics that will be used for this problem is F1-score, which is the 'average' of precision and recall.

In [83]:
def predict(X, y, parameters, activation):
    
    m = X.shape[1] 
    p = np.zeros((1,m))
    
    # Forward propagation
    probas, caches = modelForwardProp(X, parameters, activation)

    # convert probas to 0/1 predictions
    for i in range(0, probas.shape[1]):
        if probas[0,i] > 0.5:
            p[0,i] = 1
        else:
            p[0,i] = 0
    
    truePositive = []
    trueNegative = []
    falsePositive = []
    falseNegative = []
    
    for i in range (0, probas.shape[1]):
        truePositive.append((p[0,i] == 1 and y[0,i] == 1))
        trueNegative.append(p[0,i] == 0 and y[0,i] == 0)
        falsePositive.append(p[0,i] == 1 and y[0,i] == 0)
        falseNegative.append(p[0,i] == 0 and y[0,i] == 1)
    
    epsilon = 10e-8
    recall = truePositive.count(True)/(truePositive.count(True)+falseNegative.count(True)+epsilon)
    precision = truePositive.count(True)/(truePositive.count(True)+falsePositive.count(True)+epsilon)
    
    F1_score = 2*(precision*recall/(precision+recall+epsilon))
        
    return F1_score

## Gradient Descent Neural Networks

Next, let's define a function that will run the whole gradient descent optimization depending the constant parameters that we defined already.

In [77]:
def gradientDescentNN(X, Y, layerDimension, learningRate, numIterations, activation):

    np.random.seed(1)
    costs = []                  

    parameters = initializeParameters(layerDimension)
 
    
    # Loop gradient descent
    for i in range(0, numIterations):

        AL, caches = modelForwardProp(X, parameters, activation)
  
        cost = computeCost(AL, Y)
        
        grads = modelBackProp(AL, Y, caches, activation)
 
        parameters = updateParameters(parameters, grads, learningRate)
                
        # Print the cost every 100 training example
        if i % 10000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if i % 10000 == 0:
            costs.append(cost)

    return parameters

## ReLu Activation Function

After defining the steps necessary for the gradient descent to work, next we can predict the F1 score of the deep neural networks using ReLu or rectified linear unit activation function. To do this, we just need to pass 'relu' argument whenever we call the gradient descent function as follows.

In [105]:
parameters = gradientDescentNN(trainX.T, trainY.T, layerDimension, learningRate, numIterations, 'relu')

Cost after iteration 0: 0.693147
Cost after iteration 10000: 0.653012
Cost after iteration 20000: 0.653012
Cost after iteration 30000: 0.653012
Cost after iteration 40000: 0.653012
Cost after iteration 50000: 0.653012
Cost after iteration 60000: 0.653012
Cost after iteration 70000: 0.653012
Cost after iteration 80000: 0.653012
Cost after iteration 90000: 0.653012
Cost after iteration 100000: 0.653012
Cost after iteration 110000: 0.653012
Cost after iteration 120000: 0.653011
Cost after iteration 130000: 0.653010
Cost after iteration 140000: 0.653008
Cost after iteration 150000: 0.653003
Cost after iteration 160000: 0.652975
Cost after iteration 170000: 0.652561
Cost after iteration 180000: 0.158762
Cost after iteration 190000: 0.056729
Cost after iteration 200000: 0.045309


Next, we can predict the F1 score for both training set and the test set based on the Neural Networks model that has just been trained.

In [106]:
predictTrain = predict(trainX.T, trainY.T, parameters, 'relu')
predictTrain

0.9858155521477819

In [107]:
predictTest = predict(testX.T, testY.T, parameters, 'relu')
predictTest

0.9473683696986855

As can be seen above, the neural networks with ReLu activation functions has the F1 score in the training set of 98,5%, which is impressive, but the performance on the test set is just 94.7%. Let's see the comparison with the neural networks with sigmoid activation function.

## Sigmoid Activation Function

The idea of observing neural networks' F1 score with sigmoid activation function is the same with the one with relu, but now instead of 'relu', we pass 'sigmoid' as an argument whenever calling gradient descent function.

In [98]:
parameters = gradientDescentNN(trainX.T, trainY.T, layerDimension, learningRate, numIterations, 'sigmoid')

Cost after iteration 0: 0.693042
Cost after iteration 10000: 0.653010
Cost after iteration 20000: 0.653008
Cost after iteration 30000: 0.653005
Cost after iteration 40000: 0.652999
Cost after iteration 50000: 0.652987
Cost after iteration 60000: 0.652966
Cost after iteration 70000: 0.652925
Cost after iteration 80000: 0.652841
Cost after iteration 90000: 0.652654
Cost after iteration 100000: 0.652176
Cost after iteration 110000: 0.650662
Cost after iteration 120000: 0.644215
Cost after iteration 130000: 0.597069
Cost after iteration 140000: 0.270989
Cost after iteration 150000: 0.139028
Cost after iteration 160000: 0.100634
Cost after iteration 170000: 0.083415
Cost after iteration 180000: 0.073823
Cost after iteration 190000: 0.067507
Cost after iteration 200000: 0.062954


Then we can predict the F1 score in the training set as well as in the test set

In [103]:
predictTrain = predict(trainX.T, trainY.T, parameters, 'sigmoid')
predictTrain

0.9787233535712514

In [104]:
predictTest = predict(testX.T, testY.T, parameters, 'sigmoid')
predictTest

0.9624059636610349

As we can see above, the F1 score for the training set of neural networks with sigmoid function is 97,8% which is worse than the one with ReLu activation functions. However, we can see that for the test set, the neural networks with sigmoid activation function has 96.2% of F1 score, which outperforms the one with ReLu function.

Now, let's see how the neural networks with tanh activation function performs.

## tanh Activation Function

For the tanh function, the adjustment of learning rate parameter is necessary since if the learning rate is kept the same as the two activation functions before, then the gradient descent would work due to either numerical instability or because the gradient just blow up. In order to avoid this, we need to decrease the learning rate so that the learning algorithm takes a baby step at a time.

In [108]:
learningRate = 0.0025

In [109]:
parameters = gradientDescentNN(trainX.T, trainY.T, layerDimension, learningRate, numIterations, 'tanh')

Cost after iteration 0: 0.693147
Cost after iteration 10000: 0.653013
Cost after iteration 20000: 0.653012
Cost after iteration 30000: 0.653011
Cost after iteration 40000: 0.653011
Cost after iteration 50000: 0.653010
Cost after iteration 60000: 0.653008
Cost after iteration 70000: 0.653004
Cost after iteration 80000: 0.652997
Cost after iteration 90000: 0.652978
Cost after iteration 100000: 0.652913
Cost after iteration 110000: 0.652502
Cost after iteration 120000: 0.638753
Cost after iteration 130000: 0.190986
Cost after iteration 140000: 0.104909
Cost after iteration 150000: 0.081382
Cost after iteration 160000: 0.069937
Cost after iteration 170000: 0.062818
Cost after iteration 180000: 0.057885
Cost after iteration 190000: 0.054246
Cost after iteration 200000: 0.051401


Then we can predict the F1 score in the training set and the test set.

In [110]:
predictTrain = predict(trainX.T, trainY.T, parameters, 'tanh')
predictTrain

0.9823321047884256

In [111]:
predictTest = predict(testX.T, testY.T, parameters, 'tanh')
predictTest

0.9624059636610349

As we can see, the tanh activation function has a F1 score of 98.2% in the training set, and 96.2% in test set, which is comparable with the one with sigmoid activation function. However, we can't conclude that this activation function performs better than the other because the hyperparameter, which is learning rate, is different than the other two activation functions.

## Closing

The neural networks for breast cancer prediction with three different activation functions has been performed. Although all three activation functions give a very good F1 score, but perhaps it can be improved. In another project, this theme will be expanded and improved by applying different optimization algorithm other than gradient descent. The optimization algorithm that will be applied are Adam and momentum mini-batch gradient descent and then let's check whether there is improvements on the F1 score. 